In [3]:
# 이 예제는 `2. RNN 실습 - 영화평 분류`

# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import SimpleRNN, Dense, Input, Embedding
# model = Sequential()
# model.add(Input(shape=(80,))) # 입력하는 영화평의 길이를 80으로 제한, 길면 자르고, 짧으면 zero padding
# model.add(Embedding(input_dim=10000, output_dim=32))
# model.add(SimpleRNN(64))
# model.add(Dense(2, activation='softmax')) # model.add(Dense(1, activation='sigmoid'))
# model.summary()

# 이 모델의 test set accuracy는 77%

# Multi-Head Attention 구현

In [16]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Layer
import numpy as np

class MultiHeadAttention(Layer):
    def __init__(self, num_heads, key_dim):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads # 헤드의 수
        self.key_dim = key_dim # 각 헤드의 차원
        self.depth = key_dim // num_heads # 각 헤드의 깊이, shape 변경

        # head의 수 만큼 차원을 생성
        self.wq = Dense(key_dim) # q = self.wq(q) 
        self.wk = Dense(key_dim)
        self.wv = Dense(key_dim)

        self.dense = Dense(key_dim) # 출력 레이어


    def split_heads(self, x, batch_size):
        # 텐서의 모양 batch_size, seq_len, key_dim -> batch_size, num_heads, seq_len, depth
        x = tf.reshape(x, (batch_size, self.num_heads, -1, self.depth))
        return x


    # 모델을 만들어 반환
    # MultiHeadAttention(3, 32)(v, k, q) <- v, k, q는 모두 pos_enc_output이 전달
    def call(self, q, k, v):
        # 입력 텐서가 weight와 결합되어서 q, k, v로 변환
        q = self.wq(q)
        k = self.wk(k)
        v = self.wv(v)

        # ndarray.shape -> 2,3,4
        # tf.shape(텐서) -> 2,3,4 -> 2가 배치크기, shape 중에서 0번째꺼
        batch_size = tf.shape(q)[0]

        # 각 헤드로 분할
        q = self.split_heads(q, batch_size)
        k = self.split_heads(k, batch_size)
        v = self.split_heads(v, batch_size)

        # scaled dot product 계산
        matmul_qk = tf.matmul(q, k, transpose_b=True) # Q @ K.T # 행렬의 곱
        dk = tf.cast(tf.shape(k)[-1], tf.float32)
        scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

        attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
        scaled_attention = tf.matmul(attention_weights, v)

        # 각 헤드의 출력을 원래의 형태로 변환
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.key_dim))
        output = self.dense(concat_attention)

        return output

# Transformer를 이용한 영화평 분류(사용자 정의 멀티헤드어텐션 사용)

In [17]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model

inputs = layers.Input(shape=(80,))

input_embedding = layers.Embedding(input_dim=10000, output_dim=32)(inputs)
positions = tf.range(start=0, limit=80)
pos_encoding = layers.Embedding(input_dim=80, output_dim=32)(positions)
pos_enc_output = pos_encoding + input_embedding

# 직접 구현한 어텐션은 헤드의 수를 2 또는 4개로 하세요.
attention_output = MultiHeadAttention(num_heads=2, key_dim=32)(pos_enc_output, pos_enc_output, pos_enc_output) # (pos_enc_output, pos_enc_output, pos_enc_output) 로 call 함수 호출
x = layers.add([pos_enc_output, attention_output])
x = layers.BatchNormalization()(x)

ffnn = Sequential([layers.Dense(64, activation="relu"),
                   layers.Dense(32, activation="relu")])(x)
x = layers.add([ffnn, x])
x = layers.BatchNormalization()(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x) # Dropout 은 과적합 방지를 위해 사용하나 잘 사용하지 않는 추세

x = layers.Dense(64, activation="relu")(x)
x = layers.Dropout(0.1)(x) # Dropout 은 과적합 방지를 위해 사용하나 잘 사용하지 않는 추세

outputs = layers.Dense(2, activation="softmax")(x)
model = Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 80)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_4         │ (None, 80, 32)    │    320,000 │ input_layer_4[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_6 (Add)         │ (None, 80, 32)    │          0 │ embedding_4[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, None, 32)  │      4,224 │ add_6[0][0],      │
│ (MultiHeadAttentio… │                   │            │ add_6[0][0],      │
│                     │                   │            │ add_6[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_7 (Add)         │ (None, None, 32)  │          0 │ add_6[0][0],      │
│                     │                   │            │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, None, 32)  │        128 │ add_7[0][0]       │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_2        │ (None, None, 32)  │      4,192 │ batch_normalizat… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_8 (Add)         │ (None, None, 32)  │          0 │ sequential_2[0][… │
│                     │                   │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, None, 32)  │        128 │ add_8[0][0]       │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 32)        │          0 │ batch_normalizat… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_5 (Dropout) │ (None, 32)        │          0 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_18 (Dense)    │ (None, 64)        │      2,112 │ dropout_5[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_6 (Dropout) │ (None, 64)        │          0 │ dense_18[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_19 (Dense)    │ (None, 2)         │        130 │ dropout_6[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 330,914 (1.26 MB)

 Trainable params: 330,786 (1.26 MB)

 Non-trainable params: 128 (512.00 B)

# Transformer를 이용한 영화평 분류

In [18]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model

inputs = layers.Input(shape=(80,))

input_embedding = layers.Embedding(input_dim=10000, output_dim=32)(inputs)
positions = tf.range(start=0, limit=80)
pos_encoding = layers.Embedding(input_dim=80, output_dim=32)(positions)
pos_enc_output = pos_encoding + input_embedding

attention_output = layers.MultiHeadAttention(num_heads=3, key_dim=32)(pos_enc_output, pos_enc_output)
x = layers.add([pos_enc_output, attention_output])
x = layers.BatchNormalization()(x)

ffnn = Sequential([layers.Dense(64, activation="relu"),
                   layers.Dense(32, activation="relu")])(x)
x = layers.add([ffnn, x])
x = layers.BatchNormalization()(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)

x = layers.Dense(64, activation="relu")(x)
x = layers.Dropout(0.1)(x)

outputs = layers.Dense(2, activation="softmax")(x)
model = Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6       │ (None, 80)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_6         │ (None, 80, 32)    │    320,000 │ input_layer_6[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_9 (Add)         │ (None, 80, 32)    │          0 │ embedding_6[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 80, 32)    │     12,608 │ add_9[0][0],      │
│ (MultiHeadAttentio… │                   │            │ add_9[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_10 (Add)        │ (None, 80, 32)    │          0 │ add_9[0][0],      │
│                     │                   │            │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 80, 32)    │        128 │ add_10[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_3        │ (None, 80, 32)    │      4,192 │ batch_normalizat… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_11 (Add)        │ (None, 80, 32)    │          0 │ sequential_3[0][… │
│                     │                   │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 80, 32)    │        128 │ add_11[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 32)        │          0 │ batch_normalizat… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_8 (Dropout) │ (None, 32)        │          0 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_22 (Dense)    │ (None, 64)        │      2,112 │ dropout_8[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_9 (Dropout) │ (None, 64)        │          0 │ dense_22[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_23 (Dense)    │ (None, 2)         │        130 │ dropout_9[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 339,298 (1.29 MB)

 Trainable params: 339,170 (1.29 MB)

 Non-trainable params: 128 (512.00 B)

In [19]:
# model.compile(loss='binary_crossentropy', optimizer='adam',
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam',
              metrics=['accuracy'])

In [20]:
from tensorflow.keras.datasets import imdb
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=10000)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(25000,) (25000,) (25000,) (25000,)


In [21]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
X_train_pad = pad_sequences(X_train, maxlen=80, truncating='post', padding='post')
X_test_pad = pad_sequences(X_test, maxlen=80, truncating='post', padding='post')

In [22]:
%%time
model.fit(X_train_pad, y_train, epochs=10, batch_size=200)

Epoch 1/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.7555 - loss: 0.4891
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.8600 - loss: 0.3268
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8902 - loss: 0.2668
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9061 - loss: 0.2206
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.9192 - loss: 0.1782
Epoch 6/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.9353 - loss: 0.1483
Epoch 7/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.9514 - loss: 0.1166
Epoch 8/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.9642 - loss: 0.0914
Epoch 9/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9755 - loss: 0.0672
Epoch 10/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.9794 - loss: 0.0599
CPU times: user 15.3 s, sys: 2.4 s, total: 17.7 s
Wall time: 22.5 s


In [23]:
model.evaluate(X_test_pad, y_test)

782/782 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - accuracy: 0.7549 - loss: 1.5605


[1.560515284538269, 0.7549200057983398]

In [24]:
print(X_test_pad.shape)

(25000, 80)


In [25]:
import numpy as np
pred = model.predict(X_test_pad)
# pred = (pred > 0.5).astype(int)
pred = np.argmax(pred, axis=1)

782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step


In [26]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, pred))

[[ 8442  4058]
 [ 2069 10431]]


In [27]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred)

0.75492

In [ ]:
# 옵티마이저를 sgd로 바꿔보세요. accuracy: 0.57784
# 전체 단어의 개수를 1000개로 바꿔보세요. accuracy:
# 영화평의 길이를 200개로 바꿔보세요. accuracy:
# pad_sequence의 truncating과 padding을 pre로 바꿔보세요. accuracy:

In [ ]:
# 아래 택스트를 긍/부정 분류하세요.
text = "My God the actors who potrayed the VIP people cannot act. I cringed everytime they said a line. It felt like they were just reading them. Even the intonation was off. It was like when we were kids and had to read a play in class and we exagerated the intonation. Terrible, just awful."

In [ ]:
word_index = imdb.get_word_index()
# print(word_index)
# {'fawn': 34701, 'tsukino': 52006, 'nunnery': 52007,

1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
word_to_idx = {k:(v+3) for k,v in word_index.items()}
word_to_idx["<PAD>"] = 0
word_to_idx["<START>"] = 1
word_to_idx["<UNK>"] = 2  # unknown
word_to_idx["<UNUSED>"] = 3

In [ ]:
input_text = text.lower().split()
print(input_text)

['my', 'god', 'the', 'actors', 'who', 'potrayed', 'the', 'vip', 'people', 'cannot', 'act.', 'i', 'cringed', 'everytime', 'they', 'said', 'a', 'line.', 'it', 'felt', 'like', 'they', 'were', 'just', 'reading', 'them.', 'even', 'the', 'intonation', 'was', 'off.', 'it', 'was', 'like', 'when', 'we', 'were', 'kids', 'and', 'had', 'to', 'read', 'a', 'play', 'in', 'class', 'and', 'we', 'exagerated', 'the', 'intonation.', 'terrible,', 'just', 'awful.']


In [ ]:
# 단어를 숫자로 변환합니다.
# word_to_idx에 없는 단어는 2(<UNK>)로 지정하며,
# 인덱스가 10000보다 크면 3(<UNUSED>)로 지정합니다.
def encoding(review_text):
  encoded = []
  for word in review_text:
    try:
      idx = word_to_idx[word]
      if idx>10000:
        encoded.append(3)
      else:
        encoded.append(idx)
    except:
      encoded.append(2)
  return encoded

input_encoded = encoding(input_text)

In [ ]:
np.array(input_encoded)[np.newaxis, :]

array([[ 61, 558,   4, 156,  37,   2,   4,   3,  84, 566,   2,  13,   3,
          3,  36, 301,   6,   2,  12, 421,  40,  36,  71,  43, 886,   2,
         60,   4,   3,  16,   2,  12,  16,  40,  54,  75,  71, 362,   5,
         69,   8, 332,   6, 297,  11, 707,   5,  75,   3,   4,   2,   2,
         43,   2]])

In [ ]:
input_pad = pad_sequences(
    np.array(input_encoded)[np.newaxis, :],
    maxlen=80, truncating='post', padding='post')

In [ ]:
model.predict(input_pad) # LSTM을 이용한 인공신경망 모형은 부정(0)으로 분류함

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step


array([[0.8983014 , 0.10169865]], dtype=float32)